
1. EXTRACT the tweets from mongodb
-connect to a database
-query the data
2. TRANSFORM the data
-clean the text before?
-sentiment analysis
3. LOAD the data into postgres
-connect to postgres
-insert into postgress


In [1]:
#pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install psycopg2

In [3]:
import pymongo
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from pymongo import MongoClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


-Create connections to databases (check your mongodb and postgres in python notebooks (or luftdaten))

-Establish a connection to the MongoDB server

In [4]:
#MONGODB INIT
client_mongo = pymongo.MongoClient(host="localhost",port=27018)
db = client_mongo.twitter
collection=db.tweets

- Add postgres credentials

In [5]:
import cred

In [6]:
HOST = 'localhost'
PORT = '5555'
USERNAME = cred.USER
PASSWORD = cred.PASSWORD#'12345'
DB = cred.DATABASE#'sentiment_db'

- Establish a connection with Postgres

In [7]:
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}'
engine = create_engine(conn_string,echo=False)

- Select the database you want to use withing the MongoDB server and query the data

In [12]:
insert_query = "INSERT INTO greeting VALUES ('english' ,'hello!');"
engine.execute(insert_query)

- Create the function that calculates polarity_scores and returns compund score

In [13]:
engine.execute("SELECT * from greeting").fetchall()

[('english', 'hello!'), ('english', 'hello!')]

In [8]:
df_tweets = pd.DataFrame(list(collection.find()))


In [9]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   _id         100 non-null    object        
 1   tweet       100 non-null    object        
 2   tweet_id    100 non-null    int64         
 3   created_at  100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.2+ KB


In [11]:
import re

In [10]:
mentions_regex= '@[A-Za-z0-9]+'
url_regex='https?:\/\/\S+' #this will not catch all possible URLs
hashtag_regex= '#'
rt_regex= 'RT\s'

def clean_tweets(tweet):
    tweet = re.sub(mentions_regex, '', tweet)  #removes @mentions
    tweet = re.sub(hashtag_regex, '', tweet) #removes hashtag symbol
    tweet = re.sub(rt_regex, '', tweet) #removes RT to announce retweet
    tweet = re.sub(url_regex, '', tweet) #removes most URLs
    
    return tweet

In [12]:
df_tweets["tweet"] = df_tweets["tweet"].apply(clean_tweets)
df_tweets

,_id,tweet,tweet_id,created_at
0,62640ad76ae9ff712eaeaaa6,"🇺🇸🇺🇦 USA developed special drone for Ukraine, ...",1517870560805593090,2022-04-23 14:18:37
1,62640ad76ae9ff712eaeaaa7,Why is usa uk Europe openly waging war with R...,1517870245800820736,2022-04-23 14:17:21
2,62640ad76ae9ff712eaeaaa8,Support our humanitarian assistance in ukraine...,1517869930217189377,2022-04-23 14:16:06
3,62640ad76ae9ff712eaeaaa9,Something significant in relation on the Russi...,1517869908775915525,2022-04-23 14:16:01
4,62640ad76ae9ff712eaeaaaa,🇺🇳🇺🇦 UN sees 'growing evidence' of war crimes ...,1517869831202263042,2022-04-23 14:15:43
...,...,...,...,...
95,62640ad86ae9ff712eaeab05,7. The absence of compassion from the Int'l Co...,1517850686788145152,2022-04-23 12:59:38
96,62640ad86ae9ff712eaeab06,Breaking Exclusive: I've just toured Drobitsky...,1517849985357856768,2022-04-23 12:56:51
97,62640ad86ae9ff712eaeab07,1/\nWe continue to publish data on significant...,1517849937882533888,2022-04-23 12:56:40
98,62640ad86ae9ff712eaeab08,The US is planning to use tactical nuclear wea...,1517849803648303105,2022-04-23 12:56:08


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [15]:
# Make dataframe of polarity scores
pol_scores = df_tweets['tweet'].apply(analyzer.polarity_scores).apply(pd.Series)
pol_scores.head(10)

,neg,neu,pos,compound
0,0.084,0.838,0.078,-0.0516
1,0.211,0.689,0.100,-0.7667
2,0.000,0.816,0.184,0.4019
3,0.125,0.840,0.035,-0.5574
4,0.251,0.700,0.050,-0.8360
5,0.058,0.850,0.092,0.2500
6,0.298,0.585,0.117,-0.7579
7,0.386,0.376,0.238,-0.7845
8,0.054,0.731,0.216,0.6868
9,0.000,0.876,0.124,0.6597


In [16]:
df_tweets['sentiment_score']=pol_scores["compound"]

In [17]:
df_tweets.head()

,_id,tweet,tweet_id,created_at,sentiment_score
0,62640ad76ae9ff712eaeaaa6,"🇺🇸🇺🇦 USA developed special drone for Ukraine, ...",1517870560805593090,2022-04-23 14:18:37,-0.0516
1,62640ad76ae9ff712eaeaaa7,Why is usa uk Europe openly waging war with R...,1517870245800820736,2022-04-23 14:17:21,-0.7667
2,62640ad76ae9ff712eaeaaa8,Support our humanitarian assistance in ukraine...,1517869930217189377,2022-04-23 14:16:06,0.4019
3,62640ad76ae9ff712eaeaaa9,Something significant in relation on the Russi...,1517869908775915525,2022-04-23 14:16:01,-0.5574
4,62640ad76ae9ff712eaeaaaa,🇺🇳🇺🇦 UN sees 'growing evidence' of war crimes ...,1517869831202263042,2022-04-23 14:15:43,-0.8360


In [18]:
df_tweets["tweet_id"]=df_tweets["tweet_id"].values.astype(str)

In [19]:
df_tweets.iloc[:,1]

0     🇺🇸🇺🇦 USA developed special drone for Ukraine, ...
1     Why is usa uk  Europe openly waging war with R...
2     Support our humanitarian assistance in ukraine...
3     Something significant in relation on the Russi...
4     🇺🇳🇺🇦 UN sees 'growing evidence' of war crimes ...
                            ...                        
95    7. The absence of compassion from the Int'l Co...
96    Breaking Exclusive: I've just toured Drobitsky...
97    1/\nWe continue to publish data on significant...
98    The US is planning to use tactical nuclear wea...
99    When two elephants fight, it is the Ukraine gr...
Name: tweet, Length: 100, dtype: object

In [22]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   tweet            100 non-null    object        
 1   tweet_id         100 non-null    object        
 2   created_at       100 non-null    datetime64[ns]
 3   sentiment_score  100 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.2+ KB


In [20]:
del df_tweets["_id"]

In [21]:
df_tweets.to_sql('tweets', engine, if_exists='append')

In [40]:
engine.execute('''
    CREATE TABLE IF NOT EXISTS demo(
    id  VARCHAR(30),
    text TEXT,
    created_at TIMESTAMP,
    sentiment NUMERIC
);
''')

In [41]:
docs = collection.find()
for current in docs:
    print(current["created_at"])

2022-04-23 14:18:37
2022-04-23 14:17:21
2022-04-23 14:16:06
2022-04-23 14:16:01
2022-04-23 14:15:43
2022-04-23 14:15:24
2022-04-23 14:14:28
2022-04-23 14:12:43
2022-04-23 14:12:18
2022-04-23 14:12:18
2022-04-23 14:11:50
2022-04-23 14:11:22
2022-04-23 14:11:21
2022-04-23 14:10:53
2022-04-23 14:10:36
2022-04-23 14:10:32
2022-04-23 14:10:14
2022-04-23 14:09:54
2022-04-23 14:08:58
2022-04-23 14:08:50
2022-04-23 14:08:50
2022-04-23 14:07:02
2022-04-23 14:06:42
2022-04-23 14:06:38
2022-04-23 14:06:25
2022-04-23 14:05:13
2022-04-23 14:04:20
2022-04-23 14:04:15
2022-04-23 14:01:04
2022-04-23 14:00:02
2022-04-23 13:59:46
2022-04-23 13:58:21
2022-04-23 13:58:13
2022-04-23 13:57:55
2022-04-23 13:57:46
2022-04-23 13:55:54
2022-04-23 13:55:51
2022-04-23 13:55:03
2022-04-23 13:54:14
2022-04-23 13:53:46
2022-04-23 13:52:59
2022-04-23 13:52:36
2022-04-23 13:51:28
2022-04-23 13:49:31
2022-04-23 13:46:21
2022-04-23 13:46:17
2022-04-23 13:45:38
2022-04-23 13:45:33
2022-04-23 13:44:28
2022-04-23 13:42:57


In [39]:
docs = collection.find()
for current in docs:
    tweet_text = current['tweet']
    c_tweet_text=clean_tweets(tweet_text)
    tweet_id=current["tweet_id"]
    tweet_created_at=current["created_at"]
    sentiment = analyzer.polarity_scores(c_tweet_text)  # assuming your JSON docs have a text field
    score = sentiment['compound']
    query = "INSERT INTO demo VALUES (%s, %s, %s, %s);"
    engine.execute(query, (tweet_id,c_tweet_text,tweet_created_at,score))